# Downloading our Initial Dataset

In [1]:
!pip install GitPython git-pandas
!wget https://zenodo.org/record/3742962/files/enterprise-oss.tar.gz?download=1
!tar -xvf enterprise-oss.tar.gz\?download\=1

     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
  Created wheel for git-pandas: filename=git_pandas-1.2.0-py2.py3-none-any.whl size=25402 sha256=d97b5c0796fe7b5c973aafc960720cc94f3e770996fda54f40c690d0e0c8d08a
  Stored in directory: /root/.cache/pip/wheels/ea/39/c9/b213f5d8742d44a7285802862298c9f6b56a90b3695917fca9
Successfully built git-pandas
--2021-03-16 20:52:11--  https://zenodo.org/record/3742962/files/enterprise-oss.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8453437 (8.1M) [application/octet-stream]
Saving to: ‘enterprise-oss.tar.gz?download=1’

enterprise-oss.tar. 100%[===================>]   8.06M  2.68MB/s    in 3.0s    

2021-03-16 20:52:16 (2.68 MB/s) - ‘enterprise-oss.tar.gz?download=1’ saved [8453437/8453437]

README.md
enterprise_projects.txt
cohort_project_deta

In [21]:
import requests
import hashlib
import os
import git
from multiprocessing import Pool
import time
import pandas as pd
from bs4 import BeautifulSoup
from sklearn import preprocessing
import re

## Creating our intial DataFrame
We can use this dataframe to help us find the most popular enterprise oss (Open Source Software) projects. (Note: this data is from 2019)

In [3]:
# This is the shape of the data provided to us by the enterprise-oss dataset.
cols = [
        'url',
        'project_id',
        'sdtc',
        'mcpc',
        'mcve',
        'star_number',
        'commit_count',
        'files',
        'lines',
        'pull_requests',
        'github_repo_creation',
        'earliest_commit',
        'most_recent_commit',
        'committer_count',
        'author_count',
        'domainant_domain',
        'dominant_domain_committer_commits',
        'dominant_domain_author_commits',
        'dominant_domain_committers',
        'dominant_domain_authors',
        'cik',
        'fg500',
        'sec10k',
        'sec20f',
        'project_name',
        'owner_login',
        'company_name',
        'owner_company',
        'license'
        ]
        
# We are equating popularity with star_number. The thinking is the more people who 'follow' the repo, the more popular it is.
df_enterprise = pd.read_csv('enterprise_projects.txt', sep='\t', header=None, names=cols).sort_values('star_number', ascending=False).head(100)
df_enterprise.head(10)

,url,project_id,sdtc,mcpc,mcve,star_number,commit_count,files,lines,pull_requests,github_repo_creation,earliest_commit,most_recent_commit,committer_count,author_count,domainant_domain,dominant_domain_committer_commits,dominant_domain_author_commits,dominant_domain_committers,dominant_domain_authors,cik,fg500,sec10k,sec20f,project_name,owner_login,company_name,owner_company,license
9320,https://github.com/Microsoft/vscode,28947324,t,t,t,80406,55145,4210.0,1164007.0,5232,2015-09-03 20:23:38,2015-11-13 12:39:38,2019-05-31 22:11:06,831,1099,microsoft.com,43230,45274,54,323,789019.0,t,t,f,vscode,Microsoft,MICROSOFT CORPORATION,NaN,MIT
822,https://github.com/apple/swift,29261651,t,f,f,51911,85545,NaN,NaN,24225,2015-10-23 21:15:07,2010-07-17 23:50:59,2019-05-31 23:32:06,805,912,apple.com,27079,34637,80,276,320193.0,t,t,f,swift,Apple,APPLE INC,NaN,Apache-2.0
8640,https://github.com/Microsoft/TypeScript,10941284,t,t,t,51745,41276,53969.0,4923142.0,9175,2014-06-17 15:28:39,2014-07-07 22:08:16,2019-05-31 23:19:27,459,544,microsoft.com,18443,21455,29,121,789019.0,t,t,f,TypeScript,Microsoft,MICROSOFT CORPORATION,NaN,Apache-2.0
11549,https://github.com/spring-projects/spring-boot,4952344,t,f,f,39634,22099,6242.0,574034.0,3365,2012-10-19 15:02:57,2013-07-24 10:28:30,2019-05-31 22:17:55,278,741,pivotal.io,19061,17226,21,615,1574135.0,f,t,f,spring-boot,spring-projects,NaN,NaN,Apache-2.0
3895,https://github.com/ElemeFE/element,44989079,f,f,t,39620,5317,1069.0,177763.0,3662,2016-09-03 06:19:26,2016-07-27 06:15:02,2019-05-31 09:06:40,209,570,ele.me,53,63,6,8,NaN,NaN,NaN,NaN,element,ElemeFE,NaN,NaN,MIT
12579,https://github.com/Microsoft/Terminal,134861790,t,t,t,37759,512,1732.0,236460.0,172,2017-08-11 18:38:22,2018-10-08 16:09:01,2019-05-31 21:43:51,54,101,microsoft.com,249,236,8,76,789019.0,t,t,f,Terminal,Microsoft,MICROSOFT CORPORATION,NaN,MIT
3262,https://github.com/wasabeef/awesome-android-ui,14451906,t,f,f,36222,416,396.0,712.0,168,2014-12-24 01:45:03,2014-12-24 00:45:03,2018-09-19 09:45:33,48,54,cyberagent.co.jp,25,25,3,3,NaN,NaN,NaN,NaN,awesome-android-ui,wasabeef,NaN,"CyberAgent, Inc.",NaN
385,https://github.com/adobe/brackets,392,t,f,f,35448,22312,2254.0,590854.0,5459,2011-12-07 21:14:40,2011-12-07 21:20:16,2019-05-21 09:52:33,462,503,adobe.com,5624,5721,29,39,796343.0,f,t,f,brackets,adobe,ADOBE INC.,NaN,MIT
6410,https://github.com/googlesamples/android-archi...,34222729,t,f,f,33717,788,NaN,NaN,277,2016-02-05 13:42:07,2016-03-02 22:05:02,2019-03-24 10:14:57,62,79,google.com,385,513,6,15,1652044.0,f,t,f,android-architecture,googlesamples,Alphabet Inc.,NaN,Apache-2.0
5533,https://github.com/google/guava,13066443,t,f,f,33393,6667,3241.0,1026892.0,441,2014-05-29 16:23:17,2009-06-18 18:11:55,2019-05-28 15:17:22,69,367,google.com,3749,3773,12,234,1652044.0,f,t,f,guava,google,Alphabet Inc.,NaN,Apache-2.0


# Constructing Our Own Datasets
In this section, we contstruct two different datasets, one that includes statistics on the number of contributions and contributors over the last 365 days of a given project. 

The second is the number of **Stargazers**, **Commits**, and **Contributors** to a given project.

The third

In [4]:
# To construct our own dataset, we're grabbing the 20 most popular entprise github repos (as of 2019)
urls = list(df_enterprise['url'][:20])

# Helper function that gets provided to the Pool.map call in order to parallelize the downloads
def f(url): 
  print("Started cloning %s" % url)
  os.system(f"git clone --bare {url}")
  print("Finished cloning %s" % url)

start = time.time()
# Parallelize cloning the repos to reduce time.
# The number "2" provided to Pool is the number of available cores which
# you can find by calling `os.cpu_count()`
with Pool(2) as p:
  p.map(f, urls)

# For curiosity's sake, I wanted to see how much it improved the total time for this operation. As expected, reduced by half.
print(time.time() - start)

Started cloning https://github.com/Microsoft/vscode
Started cloning https://github.com/spring-projects/spring-boot
Finished cloning https://github.com/spring-projects/spring-boot
Started cloning https://github.com/ElemeFE/element
Finished cloning https://github.com/ElemeFE/element
Started cloning https://github.com/Microsoft/Terminal
Finished cloning https://github.com/Microsoft/Terminal
Started cloning https://github.com/wasabeef/awesome-android-ui
Finished cloning https://github.com/Microsoft/vscode
Started cloning https://github.com/apple/swift
Finished cloning https://github.com/wasabeef/awesome-android-ui
Started cloning https://github.com/adobe/brackets
Finished cloning https://github.com/adobe/brackets
Started cloning https://github.com/googlesamples/android-architecture
Finished cloning https://github.com/googlesamples/android-architecture
Started cloning https://github.com/google/guava
Finished cloning https://github.com/google/guava
Started cloning https://github.com/pytorch/

## Constructing our first set of DataFrames 
Each DataFrame contains statistics on the number of insertions, deletions, net insertion/deletion, and number of contriubtions that each contributor in the last $365$ days has made.

In [5]:
from gitpandas import Repository
from collections import Counter
import sys

'''Returns pandas DataFrame populated with last 365 days of commit history.
Data includes insertions, deletions, net insertion/deletion, and total commits
per individual contributor.
'''
def get_repo_stats(repo):
  try:
    print(f'Starting {repo.git_dir}', file=sys.stderr)
    
    # Some repos use master or main (try both), styleguide doesn't use main or master
    try:
      b = "master" if repo.git_dir != 'styleguide.git' else 'gh-pages'
      df_repo = repo.commit_history(branch=b, days=365)  
    except Exception:
      df_repo = repo.commit_history(branch='main', days=365) 
    
    # Get the total number of contributions each person has made
    counts = Counter(df_repo.author)

    # Group by author and sum the columns per author
    df_group = df_repo.groupby('author')
    df_groupsum = df_group.sum()
    df_groupsum['contributions'] = [counts[x] for x in list(df_group.indices.keys())]
    
    # Write out the dataframe as a csv
    with open(f'{repo.git_dir}.csv', mode='w') as file:
      file.write(df_groupsum.to_csv())
    print(f"Finished {repo.git_dir}", file=sys.stderr)
    
    return df_groupsum

  except Exception:
    # Likely we'll encounter this error if the branch is not master or main.
    return f"Error on repository {repo}"

In [6]:
import warnings
# Get all directories in the present working directory that end in "git"
repos = [Repository(x) for x in list(os.walk('.'))[0][1] if x[-3:] == 'git']

# Remove deprecation warnings from pandas to prevent clogging stdout
warnings.filterwarnings('ignore', category=UserWarning)

# Parallelize the calls to reduce time
start = time.time()
with Pool(2) as p:
  x = list(p.map(get_repo_stats, repos))
print(time.time() - start)

Starting Terminal.git
Starting fastText.git
Finished fastText.git
Starting material2.git
Finished material2.git
Starting guava.git
Finished Terminal.git
Starting echarts.git
Finished guava.git
Starting kotlin.git
Finished echarts.git
Starting android-architecture.git
Finished android-architecture.git
Starting awesome-android-ui.git
Finished awesome-android-ui.git
Starting styleguide.git
Finished styleguide.git
Starting Rocket.Chat.git
Finished Rocket.Chat.git
Starting element.git
Finished element.git
Starting vscode.git
Finished kotlin.git
Starting flow.git
Finished flow.git
Starting spark.git
Finished spark.git
Starting spring-boot.git
Finished vscode.git
Starting brackets.git
Finished brackets.git
Starting swift.git
Finished spring-boot.git
Starting pytorch.git
Finished pytorch.git
Starting fastjson.git
Finished fastjson.git
Finished swift.git
Starting TypeScript.git
Finished TypeScript.git


1159.36412358284


In [7]:
# Create a tarball of all the newly created CSVs so that we can import them into our other workbooks
!tar czvf data.tar.gz *.csv

android-architecture.git.csv
awesome-android-ui.git.csv
brackets.git.csv
echarts.git.csv
element.git.csv
fastjson.git.csv
fastText.git.csv
flow.git.csv
guava.git.csv
kotlin.git.csv
material2.git.csv
pytorch.git.csv
Rocket.Chat.git.csv
spark.git.csv
spring-boot.git.csv
styleguide.git.csv
swift.git.csv
Terminal.git.csv
TypeScript.git.csv
vscode.git.csv


## Constructing our second set of data
This singular DataFrame includes the total number of Stargazers, Commits, and Contributors to a specific project. Additional information includes the project's company.

In [85]:
token = 'REDACTED'
def contributor_count(owner, repo):
  """
  Although there is a GitHub API call we can use to get the total number of contributors,
  there is a disparity between the API result and the value on the repositories 
  main webpage. Since the webpages' values seem more representative of the actual state
  of the project, we're opting to use that.
  """
  response = requests.get(f'https://github.com/{owner}/{repo}', headers={'Authorization': f'token {token}'})

  if response.status_code != 200:
    raise requests.HTTPError(response.text, response.status_code, response.headers)

  bs = BeautifulSoup(response.text, 'lxml')

  try:
    """
    We need to scrape for the span that contains the number of contributors.
    Since the text "contributors" is only in the parent, we need to query the
    parent object to make sure we grab the "Contributors" value instead of 
    something else
    """
    a = [x for x in bs.find_all('span', class_='Counter ') if 'contributors' in x.parent['href']][0]
  except Exception:
    IndexError("No contributor found for repo {owner}/{repo}")

  if a is None:
    raise RuntimeError(f"No contributor count for repo {owner}/{repo}")

  return a.text

def commit_count(owner, repo):
  """
  Since GitHub doesn't actually provide the total number of commits in any API call
  we can instead use a query parameter to set the page size for each API call to 1
  Since GH links us the last entry for pagination purposes, we can just look at the
  page number of the last commit in the repo to find the total number of commits
  """
  try:
    response = requests.get(f'https://api.github.com/repos/{owner}/{repo}/commits?per_page=1', headers={'Authorization': f'token {token}'})

    # Use Regex to get the last commit
    return re.search('\d+$', response.links['last']['url']).group()
  except Exception:
    raise requests.HTTPError(f"Could not find last link for repo {owner}/{repo}", response)
 
def stargazers_count(owner, repo):
  # We can directly get the number of followers from the API call
  try:
    response = requests.get(f'https://api.github.com/repos/{owner}/{repo}', headers={'Authorization': f'token {token}'})
    return response.json()['stargazers_count']
  except Exception:
    RuntimeError(f"No stargazers for repo {owner}/{repo}", response)

In [ ]:
# Using the 30 most popular enterprise-oss projects (as of 2019) get the number of followers, contributors, and commits (as of NOW)
df_vis = pd.DataFrame(map(lambda o, r: [r, o, stargazers_count(o,r), contributor_count(o,r), commit_count(o,r)], df_enterprise['owner_login'][:30], df_enterprise['project_name'][:30]), columns=['Repo','Company','Followers','Contributors','Commits']).set_index('Repo')

# Clean data to recognize integers
df_vis['Commits'] = df_vis['Commits'].str.replace(',','').astype('int64')
df_vis['Contributors'] = df_vis['Contributors'].str.replace(',','').astype('int64')

# Use a min-max scaler for our commit count, so that when we graph, the size of each
# marker will be easier to compare total commit counts. 
# (MinMaxScaler provided the best results)
x = df_vis[['Commits']]
scaler = preprocessing.MinMaxScaler()
df_vis["Commits_Normalized"] = scaler.fit_transform(x)
df_vis

,Company,Followers,Contributors,Commits,Commits_Normalized
Repo,,,,,
vscode,Microsoft,112889,1377,78430,0.660045
swift,Apple,55390,857,118630,1.000000
TypeScript,Microsoft,69355,564,31874,0.266340
spring-boot,spring-projects,53975,805,31276,0.261283
element,ElemeFE,49424,545,4441,0.034351
Terminal,Microsoft,72567,274,2066,0.014266
awesome-android-ui,wasabeef,40051,69,452,0.000617
brackets,adobe,33181,372,17846,0.147711
android-architecture,googlesamples,38386,50,691,0.002638


In [ ]:
# Create tarball of this csv so that we can import that into our other notebook.
with open('commit-visualization.csv', mode='w') as file:
  file.write(df_vis.to_csv())

!tar -czvf commit-visualization.tar.gz commit-visualization.csv

commit-visualization.csv


## Constructing our third dataset
This DataFrame is a deep copy of the original `df_enterprise`, but we add additional information fromt the GitHub API. Specifically, we include aggreate weekly commit information for each repository based on the last year.

In [35]:
def get_zipped_uris():
  """
  Returns a tuple containing the repo's owner and the project name
  We use this information to access the corret URI on the GH API
  """
  return zip(df_commits['owner_login'], df_commits['project_name'])

def get_project_languages(a):
  """
  Gets the languages associated with the project
  """
  response = requests.get(f'https://api.github.com/repos/{a[0]}/{a[1]}/languages',
                          headers = {'Accept': 'application/vnd.github.mercy-preview',
                                     'Authorization': f'token {token}'})

  if response.status_code != 200:
    raise requests.HTTPError(f"Repository '{a[0]}/{a[1]}' returned a non-200" + 
                             " when searching for languages", response.status_code,
                             response.text)

  # We're running these requests in parallel, so let's give a delay
  time.sleep(.2)

  return response.json()

def get_project_topics(a):
  """
  Gets the topics associated with the project
  """
  response = requests.get(f'https://api.github.com/repos/{a[0]}/{a[1]}/topics',
                          headers = {'Accept': 'application/vnd.github.mercy-preview',
                                     'Authorization': f'token {token}'})

  if response.status_code != 200:
    raise requests.HTTPError(f"Repository '{a[0]}/{a[1]}' returned a non-200" + 
                             " when searching for topics", response.status_code,
                             response.text)

  # We're running these requests in parallel, so let's give a delay
  time.sleep(.2)

  return response.json()

def get_project_commit_activity(a):
  """
  Gets the weekly commit activity associated with the project for the last year
  """
  response = requests.get(f'https://api.github.com/repos/{a[0]}/{a[1]}/stats/commit_activity',
                          headers = {'Accept': 'application/vnd.github.mercy-preview',
                                     'Authorization': f'token {token}'})

  # If the data we're looking for isn't cached, wait for the lookup to happen
  # and get the cached result
  if response.status_code == 202:
    time.sleep(5)
    response = requests.get(f'https://api.github.com/repos/{a[0]}/{a[1]}/stats/commit_activity',
                          headers = {'Accept': 'application/vnd.github.mercy-preview',
                                     'Authorization': f'token {token}'})

  if response.status_code != 200:
    raise requests.HTTPError(f"Repository '{a[0]}/{a[1]}' returned a non-200" + 
                             " when searching for commit activity", response.status_code,
                             response.text)

  # We're running these requests in parallel, so let's give a delay
  time.sleep(.2)

  return response.json()

In [37]:
df_commits = df_enterprise.copy(deep=True)

start = time.time()
# Get the languages, topics, and last 52 weeks of commit activity
with Pool(2) as p:
  languages = p.map(get_project_languages, get_zipped_uris())
  topics = p.map(get_project_topics, get_zipped_uris())
  commit_activity = p.map(get_project_commit_activity, get_zipped_uris())

print(time.time() - start)

# Processes the commit data and creates mappings from week to total weekly commit
project_commits = [dict([(pd.to_datetime(y['week'], unit='s', origin='unix').date(), y['total']) for y in x]) for x in commit_activity]

98.02564883232117


In [84]:
# Resets the index so we can join all our new data together, and then sets the index to the project_name
df_commits_final = df_commits.reset_index().join(pd.DataFrame(topics)).join(pd.DataFrame(languages)).join(pd.DataFrame(a)).drop(columns='index').set_index('project_name')
df_commits_final.to_csv('master_df.csv')
df_commits_final['fg500'].dropna()

project_name
vscode                          t
swift                           t
TypeScript                      t
spring-boot                     f
Terminal                        t
brackets                        f
android-architecture            f
guava                           f
echarts                         f
styleguide                      f
material2                       f
polymer                         f
istio                           f
CNTK                            t
metasploit-framework            f
calculator                      t
source-code-pro                 f
monaco-editor                   t
python-fire                     f
minikube                        f
apollo                          f
MS-DOS                          t
magenta                         f
plaid                           f
dotnet                          t
core                            t
android-UniversalMusicPlayer    f
tfjs                            f
react-native-navigation         f
a